from:https://blog.csdn.net/qq_25491201/article/details/51065547

In [ ]:
import numpy as np
import cv2
def detect(image):
    # 把图像从RGB装换成灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 使用Scharr操作（指定使用ksize = -1）构造灰度图在水平和竖直方向上的梯度幅值表示。
    gradX = cv2.Sobel(gray, ddepth = cv2.cv.CV_32F, dx = 1, dy = 0, ksize = -1)
    gradY = cv2.Sobel(gray, ddepth = cv2.cv.CV_32F, dx = 0, dy = 1, ksize = -1)
    #Scharr操作后，从x的梯度减去y的梯度
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    blurred = cv2.blur(gradient, (9, 9))
    (_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)

#上面的操作后发现有一些条形码的竖杠之间存在一些缝隙，并使我们的算法能检测到斑点区域，
#我们进行一些基本的形态学操作#我们首先使用cv2.getStructuringElement构造一个长方形内核。
#这个内核的宽度大于长度，因此我们可以消除条形码中垂直条之间的缝隙。
# 这里进行形态学操作，将上一步得到的内核应用到我们的二值图中，以此来消除竖杠间的缝隙。
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    #我们发现还是存在一些小斑点，这时我们可以使用opencv里面的腐蚀和膨胀来处理他们，来去除白色的斑点
    closed = cv2.erode(closed, None, iterations = 4)
    closed = cv2.dilate(closed, None, iterations = 4)
    # 接下来我们来找轮廓
    (cnts, _) = cv2.findContours(closed.copy(), 
                                 cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #如果没找到则返回为空
    if len(cnts) == 0:
        return None
    
    #找到了就通过面积来排序，并计算旋转角# 给最大的轮廓找到边框
    c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
    rect = cv2.minAreaRect(c)
    #box(里面是ndarry数组，包含了4个顶点的位置)
    box = np.int0(cv2.cv.BoxPoints(rect))
    # 返回box
    return box


In [9]:
import numpy as np
import cv2

def detect(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # compute the Scharr gradient magnitude representation of the images
    # in both the x and y direction
    gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
    gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
    
    #cv2.imshow("gradY", gradY)
    #cv2.waitKey(0)
    # subtract the y-gradient from the x-gradient
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)

    # blur and threshold the image
    blurred = cv2.blur(gradient, (9, 9))
    (_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)

    # construct a closing kernel and apply it to the thresholded image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # perform a series of erosions and dilations
    closed = cv2.erode(closed, None, iterations = 4)
    closed = cv2.dilate(closed, None, iterations = 4)

    # find the contours in the thresholded image
    (image,cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cv2.imshow("image", image)
    cv2.waitKey(0)
    # if no contours were found, return None
    if len(cnts) == 0:
        return None

    # otherwise, sort the contours by area and compute the rotated
    # bounding box of the largest contour
    c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
    rect = cv2.minAreaRect(c)
    box = np.int0(cv2.boxPoints(rect))

    # return the bounding box of the barcode
    return box

img = cv2.imread("./2018-09-25 13-15.png", cv2.IMREAD_COLOR)
#cv2.imshow("image", img)
#cv2.waitKey(0)
box = detect(img)
print(box)

#CutImage=image[box[0]:box[1],box[2]:box[3]]
#cv2.imshow("CutImage", CutImage)
#cv2.waitKey(0)

[[ 41 141]
 [ 33 141]
 [ 33 131]
 [ 41 131]]


In [9]:
import numpy as np
import cv2
#1，先将读入的摄像头frame转换成灰度图：
#image_path = "./2018-09-25 13-15.png"
img = cv2.imread("./2018-09-25 13-15.png", cv2.IMREAD_COLOR)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#2，使用opencv自带的Sobel算子进行过滤：

gradX = cv2.Sobel(gray, cv2.CV_32F, 1, 0,-1)
gradY = cv2.Sobel(gray, cv2.CV_32F, 0, 1,-1)

#具体参数可参考：http://blog.csdn.net/sunny2038/article/details/9170013

#3，将过滤得到的X方向像素值减去Y方向的像素值：

gradient = cv2.subtract(gradX, gradY)

#4，先缩放元素再取绝对值，最后转换格式为8bit型
#cv2.imshow("image", gradient)
#cv2.waitKey(0)
gradient = cv2.convertScaleAbs(gradient)

#5，均值滤波取二值化：

blurred = cv2.blur(gradient, (9, 9))
(_, thresh) = cv2.threshold(blurred, 120, 120, cv2.THRESH_BINARY)
cv2.imshow("image", gradient)
cv2.waitKey(0)
print(gradient.shape)

#for i in range(blurred.height):
#  for j in range(blurred.width):
#    iTmp[i,j] = 255 - blurred[i,j]
#cv2.imwrite( "./00.jpg",iTmp)
#6，腐蚀和膨胀的函数：

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

closed = cv2.erode(closed, None, iterations = 4)
closed = cv2.dilate(closed, None, iterations = 4)
cv2.imshow("image", closed)
cv2.waitKey(0)
#7，找到边界findContours函数

binary,cnts,hierarchy = cv2.findContours(blurred.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

#8，计算出包围目标的最小矩形区域：
print(cnts)
c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))
print(box)

(432, 428)
[array([[[ 14,   0]],

       [[ 14,   5]],

       [[ 13,   6]],

       [[ 13,  30]],

       [[ 12,  31]],

       [[  9,  31]],

       [[  8,  32]],

       [[  5,  32]],

       [[  5,  33]],

       [[  3,  35]],

       [[  3,  49]],

       [[  4,  50]],

       [[  7,  50]],

       [[  8,  51]],

       [[ 12,  51]],

       [[ 13,  52]],

       [[ 13,  54]],

       [[  9,  58]],

       [[  8,  58]],

       [[  8,  60]],

       [[  6,  62]],

       [[  5,  62]],

       [[  5,  63]],

       [[  4,  64]],

       [[  4,  69]],

       [[  5,  70]],

       [[  5,  72]],

       [[  6,  73]],

       [[ 12,  73]],

       [[ 13,  74]],

       [[ 13, 107]],

       [[ 12, 108]],

       [[ 12, 109]],

       [[ 11, 110]],

       [[  3, 110]],

       [[  3, 116]],

       [[ 12, 116]],

       [[ 13, 117]],

       [[ 13, 123]],

       [[ 14, 124]],

       [[ 14, 128]],

       [[ 13, 129]],

       [[ 13, 131]],

       [[ 12, 132]],

       [[ 11, 131]],

In [4]:
if box != None:
    # 这下面的3步得到扫描区域，扫描区域要比检测出来的位置要大
    min = np.min(box, axis=0)
    max = np.max(box, axis=0)

    roi = img[min[1] - 10:max[1] + 10, min[0] - 10:max[0] + 10]
    print roi.shape
    # 把区域里的二维码传换成RGB，并把它转换成pil里面的图像，因为zbar得调用pil里面的图像，而不能用opencv的图像
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img).convert('L')
    width, height = pil.size
    raw = pil.tostring()

(10, 10, 3)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  """Entry point for launching an IPython kernel.


NameError: name 'Image' is not defined

In [7]:
# -*- coding=utf-8 -*-
import os
import cv2
import numpy as np
import copy
img_path='images'
img_result='results'
def reshape_image(image):
    '''归一化图片尺寸：短边400，长边不超过800，短边400，长边超过800以长边800为主'''
    width,height=image.shape[1],image.shape[0]
    min_len=width
    scale=width*1.0/400
    new_width=400
    new_height=int(height/scale)
    if new_height>800:
        new_height=800
        scale=height*1.0/800
        new_width=int(width/scale)
    out=cv2.resize(image,(new_width,new_height))
    return out
def detecte(image):
    '''提取所有轮廓'''
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    _,gray=cv2.threshold(gray,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    img,contours,hierachy=cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    return image,contours,hierachy

def compute_1(contours,i,j):
    '''最外面的轮廓和子轮廓的比例'''
    area1 = cv2.contourArea(contours[i])
    area2 = cv2.contourArea(contours[j])
    if area2==0:
        return False
    ratio = area1 * 1.0 / area2
    if abs(ratio - 49.0 / 25):
        return True
    return False
def compute_2(contours,i,j):
    '''子轮廓和子子轮廓的比例'''
    area1 = cv2.contourArea(contours[i])
    area2 = cv2.contourArea(contours[j])
    if area2==0:
        return False
    ratio = area1 * 1.0 / area2
    if abs(ratio - 25.0 / 9):
        return True
    return False
def compute_center(contours,i):
    '''计算轮廓中心点'''
    M=cv2.moments(contours[i])
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    return cx,cy
def detect_contours(vec):
    '''判断这个轮廓和它的子轮廓以及子子轮廓的中心的间距是否足够小'''
    distance_1=np.sqrt((vec[0]-vec[2])**2+(vec[1]-vec[3])**2)
    distance_2=np.sqrt((vec[0]-vec[4])**2+(vec[1]-vec[5])**2)
    distance_3=np.sqrt((vec[2]-vec[4])**2+(vec[3]-vec[5])**2)
    if sum((distance_1,distance_2,distance_3))/3<3:
        return True
    return False
def juge_angle(rec):
    '''判断寻找是否有三个点可以围成等腰直角三角形'''
    if len(rec)<3:
        return -1,-1,-1
    for i in range(len(rec)):
        for j in range(i+1,len(rec)):
            for k in range(j+1,len(rec)):
                distance_1 = np.sqrt((rec[i][0] - rec[j][0]) ** 2 + (rec[i][1] - rec[j][1]) ** 2)
                distance_2 = np.sqrt((rec[i][0] - rec[k][0]) ** 2 + (rec[i][1] - rec[k][1]) ** 2)
                distance_3 = np.sqrt((rec[j][0] - rec[k][0]) ** 2 + (rec[j][1] - rec[k][1]) ** 2)
                if abs(distance_1-distance_2)<5:
                    if abs(np.sqrt(np.square(distance_1)+np.square(distance_2))-distance_3)<5:
                        return i,j,k
                elif abs(distance_1-distance_3)<5:
                    if abs(np.sqrt(np.square(distance_1)+np.square(distance_3))-distance_2)<5:
                        return i,j,k
                elif abs(distance_2-distance_3)<5:
                    if abs(np.sqrt(np.square(distance_2)+np.square(distance_3))-distance_1)<5:
                        return i,j,k
    return -1,-1,-1
def find(image,image_name,contours,hierachy,root=0):
    '''找到符合要求的轮廓'''
    rec=[]
    for i in range(len(hierachy)):
        child = hierachy[i][2]
        child_child=hierachy[child][2]
        if child!=-1 and hierachy[child][2]!=-1:
            if compute_1(contours, i, child) and compute_2(contours,child,child_child):
                cx1,cy1=compute_center(contours,i)
                cx2,cy2=compute_center(contours,child)
                cx3,cy3=compute_center(contours,child_child)
                if detect_contours([cx1,cy1,cx2,cy2,cx3,cy3]):
                    rec.append([cx1,cy1,cx2,cy2,cx3,cy3,i,child,child_child])
    '''计算得到所有在比例上符合要求的轮廓中心点'''
    i,j,k=juge_angle(rec)
    if i==-1 or j== -1 or k==-1:
        return
    ts = np.concatenate((contours[rec[i][6]], contours[rec[j][6]], contours[rec[k][6]]))
    rect = cv2.minAreaRect(ts)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    result=copy.deepcopy(image)
    cv2.drawContours(result, [box], 0, (0, 0, 255), 2)
    cv2.drawContours(image,contours,rec[i][6],(255,0,0),2)
    cv2.drawContours(image,contours,rec[j][6],(255,0,0),2)
    cv2.drawContours(image,contours,rec[k][6],(255,0,0),2)
    cv2.imshow('img',image)
    cv2.waitKey(0)
    cv2.imshow('img',result)
    cv2.waitKey(0)
    path=os.path.join(img_result,image_name)
    cv2.imwrite(path,result)
    return
if __name__ == '__main__':
    img_path = "./"
    files=os.listdir(img_path)
    for file in files:
        image=cv2.imread(os.path.join(img_path,file))
        image=reshape_image(image)
        cv2.imshow('img', image)
        cv2.waitKey(0)
        image,contours,hierachy=detecte(image)
        find(image,file,contours,np.squeeze(hierachy))

AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:
# -*- coding=utf-8 -*-
import os
import cv2
import numpy as np
import copy
img_path='images'
img_result='results'
def reshape_image(image):
    '''归一化图片尺寸：短边400，长边不超过800，短边400，长边超过800以长边800为主'''
    width,height=image.shape[1],image.shape[0]
    min_len=width
    scale=width*1.0/400
    new_width=400
    new_height=int(height/scale)
    if new_height>800:
        new_height=800
        scale=height*1.0/800
        new_width=int(width/scale)
    out=cv2.resize(image,(new_width,new_height))
    return out
def detecte(image):
    '''提取所有轮廓'''
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #_,gray=cv2.threshold(gray,100,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    _,gray=cv2.threshold(gray,160,255,cv2.THRESH_BINARY_INV)
    cv2.imshow('img', gray)
    cv2.waitKey(0)
    img,contours,hierachy=cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    return image,contours,hierachy

def compute_1(contours,i,j):
    '''最外面的轮廓和子轮廓的比例'''
    area1 = cv2.contourArea(contours[i])
    area2 = cv2.contourArea(contours[j])
    if area2==0:
        return False
    ratio = area1 * 1.0 / area2
    if abs(ratio - 49.0 / 25):
        return True
    return False
def compute_2(contours,i,j):
    '''子轮廓和子子轮廓的比例'''
    area1 = cv2.contourArea(contours[i])
    area2 = cv2.contourArea(contours[j])
    if area2==0:
        return False
    ratio = area1 * 1.0 / area2
    if abs(ratio - 25.0 / 9):
        return True
    return False
def compute_center(contours,i):
    '''计算轮廓中心点'''
    M=cv2.moments(contours[i])
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    return cx,cy
def detect_contours(vec):
    '''判断这个轮廓和它的子轮廓以及子子轮廓的中心的间距是否足够小'''
    distance_1=np.sqrt((vec[0]-vec[2])**2+(vec[1]-vec[3])**2)
    distance_2=np.sqrt((vec[0]-vec[4])**2+(vec[1]-vec[5])**2)
    distance_3=np.sqrt((vec[2]-vec[4])**2+(vec[3]-vec[5])**2)
    if sum((distance_1,distance_2,distance_3))/3<3:
        return True
    return False
def juge_angle(rec):
    '''判断寻找是否有三个点可以围成等腰直角三角形'''
    if len(rec)<3:
        return -1,-1,-1
    for i in range(len(rec)):
        for j in range(i+1,len(rec)):
            for k in range(j+1,len(rec)):
                distance_1 = np.sqrt((rec[i][0] - rec[j][0]) ** 2 + (rec[i][1] - rec[j][1]) ** 2)
                distance_2 = np.sqrt((rec[i][0] - rec[k][0]) ** 2 + (rec[i][1] - rec[k][1]) ** 2)
                distance_3 = np.sqrt((rec[j][0] - rec[k][0]) ** 2 + (rec[j][1] - rec[k][1]) ** 2)
                if abs(distance_1-distance_2)<5:
                    if abs(np.sqrt(np.square(distance_1)+np.square(distance_2))-distance_3)<5:
                        return i,j,k
                elif abs(distance_1-distance_3)<5:
                    if abs(np.sqrt(np.square(distance_1)+np.square(distance_3))-distance_2)<5:
                        return i,j,k
                elif abs(distance_2-distance_3)<5:
                    if abs(np.sqrt(np.square(distance_2)+np.square(distance_3))-distance_1)<5:
                        return i,j,k
    return -1,-1,-1
def find(image,image_name,contours,hierachy,root=0):
    '''找到符合要求的轮廓'''
    rec=[]
    #print("hierachy len:", len(hierachy))
    for i in range(len(hierachy)):
        child = hierachy[i][2]
        child_child=hierachy[child][2]
        #print("child:",child,"hierachy[child][2]:",hierachy[child][2])
        if child!=-1 and hierachy[child][2]!=-1:#子类和子类的子类都存在
            if compute_1(contours, i, child) and compute_2(contours,child,child_child):
                cx1,cy1=compute_center(contours,i)
                cx2,cy2=compute_center(contours,child)
                cx3,cy3=compute_center(contours,child_child)
                print "contours", contours[i], "i", i, "child", child, "child_child",child_child
                print "1:", cx1, cy1, "2:", cx2, cy2, "3:", cx3, cy3 
                if detect_contours([cx1,cy1,cx2,cy2,cx3,cy3]):
                    rec.append([cx1,cy1,cx2,cy2,cx3,cy3,i,child,child_child])
    '''计算得到所有在比例上符合要求的轮廓中心点'''
    i,j,k=juge_angle(rec)
    print "i=", i, "j=", j,"k=", k
    if i==-1 or j== -1 or k==-1:
        return
    ts = np.concatenate((contours[rec[i][6]], contours[rec[j][6]], contours[rec[k][6]]))
    rect = cv2.minAreaRect(ts)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    result=copy.deepcopy(image)
    cv2.drawContours(result, [box], 0, (0, 0, 255), 2)
    cv2.drawContours(image,contours,rec[i][6],(255,0,0),2)
    cv2.drawContours(image,contours,rec[j][6],(255,0,0),2)
    cv2.drawContours(image,contours,rec[k][6],(255,0,0),2)
    cv2.imshow('img',image)
    cv2.waitKey(0)
    cv2.imshow('img',result)
    cv2.waitKey(0)
    path=os.path.join(img_result,image_name)
    cv2.imwrite(path,result)
    return
if __name__ == '__main__':
    image=cv2.imread("./qrcode.png")
    print(image.shape)
    image=reshape_image(image)
    print(image.shape)

    image2,contours,hierachy=detecte(image)
    #print("contours:" , hierachy)
    cv2.drawContours(image2,contours[59],-1,(0,0,255),3)  
  
    cv2.imshow("img", image2)  
    cv2.waitKey(0)
    find(image,file,contours,np.squeeze(hierachy))

(441, 441, 3)
(400, 400, 3)
contours [[[ 33 310]]

 [[ 33 366]]

 [[ 89 366]]

 [[ 89 310]]] i 1 child 2 child_child 3
1: 61 338 2: 60 338 3: 60 338
contours [[[310  33]]

 [[310  89]]

 [[366  89]]

 [[366  33]]] i 11 child 12 child_child 13
1: 338 61 2: 338 60 3: 338 60
contours [[[33 33]]

 [[33 89]]

 [[89 89]]

 [[89 33]]] i 115 child 116 child_child 117
1: 61 61 2: 60 60 3: 60 60
i= 0 j= 1 k= 2


AttributeError: type object 'file' has no attribute 'startswith'

In [3]:
    image=cv2.imread("./123.png")
    print(image.shape)
    #image=reshape_image(image)

    image2,contours,hierachy=detecte(image)
    print("contours:" , contours)
    print("hierachy:", hierachy) 
    print(contours[1])
    cv2.drawContours(image2,contours[1],-1,(0,0,255),3)  
  
    cv2.imshow("img", image2)  
    cv2.waitKey(0)

(475, 716, 3)
('contours:', [array([[[ 74,  55]],

       [[ 74, 270]],

       [[300, 270]],

       [[300,  55]]], dtype=int32), array([[[ 81,  63]],

       [[ 82,  62]],

       [[292,  62]],

       [[293,  63]],

       [[293, 262]],

       [[292, 263]],

       [[ 82, 263]],

       [[ 81, 262]]], dtype=int32), array([[[ 99, 113]],

       [[ 99, 186]],

       [[204, 186]],

       [[204, 113]]], dtype=int32), array([[[106, 121]],

       [[107, 120]],

       [[196, 120]],

       [[197, 121]],

       [[197, 178]],

       [[196, 179]],

       [[107, 179]],

       [[106, 178]]], dtype=int32), array([[[409,  34]],

       [[409, 342]],

       [[642, 342]],

       [[642,  34]]], dtype=int32), array([[[416,  42]],

       [[417,  41]],

       [[634,  41]],

       [[635,  42]],

       [[635, 334]],

       [[634, 335]],

       [[417, 335]],

       [[416, 334]]], dtype=int32), array([[[451, 173]],

       [[451, 301]],

       [[600, 301]],

       [[600, 173]]], dtype=i

-1

In [75]:
grid_num = 29
image=cv2.imread("./2018-09-25 13-15.png")
(H,W,_) = image.shape
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#_,gray=cv2.threshold(gray,100,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
_,gray=cv2.threshold(gray,160,255,cv2.THRESH_BINARY_INV)
#cv2.imshow('img', gray)
#cv2.waitKey(0)

step = float(W)/grid_num
print step

image_new = np.zeros((image.shape)).astype("uint8")
image_new[:, :, 0] = gray
image_new[:, :, 1] = gray
image_new[:, :, 2] = gray

for i in range (grid_num):
    cv2.line(image_new, (int(round(i*step)), 0), (int(round(i*step)),H), (0, 0, 255), 1)
    cv2.line(image_new, (0, int(round(i*step))), (W,int(round(i*step))), (0, 0, 255), 1)

grid_data = np.zeros((grid_num, grid_num, int(step), int(step)))
min_grid_data = np.zeros((grid_num, grid_num))

for i in range (grid_num):
    for j in range (grid_num):
        for k in range (int(step)):
            for l in range (int(step)):
                grid_data[i,j,k,l] = gray[int(round(i*step)) + k, int(round(j*step)) + l]

for i in range (grid_num):
    for j in range (grid_num):
        cnt = 0
        for k in range (int(step)):

            for l in range (int(step)):  
                if grid_data[i,j,k,l] == 255:
                    cnt = cnt + 1
        #print cnt, i    
        if (cnt >= 10):
            min_grid_data[i,j] = 1
        else:
            min_grid_data[i,j] = 0


qr_infor = [int(min_grid_data[8,0]), 
            int(min_grid_data[8,1]),
            int(min_grid_data[8,2]),
            int(min_grid_data[8,3]),
            int(min_grid_data[8,4]),
            int(min_grid_data[8,5]),
            
            int(min_grid_data[8,7]),
            int(min_grid_data[8,8]),
            
            int(min_grid_data[7,8]),
            int(min_grid_data[5,8]),
            int(min_grid_data[4,8]),
            int(min_grid_data[3,8]),
            int(min_grid_data[2,8]),
            int(min_grid_data[1,8]),
            int(min_grid_data[0,8])                       
           ]
print qr_infor
cv2.imshow("Canvas", image_new)
cv2.waitKey(0)

13.724137931
[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]


-1

In [54]:
print gray[14:27,14:27]
print gray[14:27,0:13]
print gray.shape

[[0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[  0 255 255 255 255 255 255 255 255 255   0   0   0]
 [  0 255 255 255 255 255 255 255 255 255   0   0   0]
 [  0 255 255 255 255 255 255 255 255 255 255   0   0]
 [255 255 255 255 255 255 255 255 255 255 255   0   0]
 [255 255 255 255 255 255 255 255 255 255 255   0   0]
 [255 255 255 255 255 255 255 255 255 255 255 255   0]
 [255 255 255 255 255 255 255 255 255 255 255 255   0]
 [255 255 255 255 255 255 255 255 255 255 255 255   0]
 [255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255

In [58]:
print grid_data[9,0,:,:]

[[   0.    0.  255.    0.  255.  255.  255.  255.    0.    0.    0.    0.
     0.]
 [   0.  255.  255.  255.  255.  255.  255.  255.  255.    0.    0.    0.
     0.]
 [ 255.  255.  255.  255.  255.  255.  255.  255.  255.  255.    0.    0.
     0.]
 [ 255.  255.  255.  255.  255.  255.  255.  255.  255.    0.    0.    0.
     0.]
 [   0.  255.  255.  255.  255.  255.  255.  255.  255.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.  255.  255.  255.  255.  255.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.  255.  255.  255.  255.  255.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.    0.  255.  255.  255.    0.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.    0.    0.  255.  255.  255.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.    0.    0.    0.  255.  255.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  255.    0.    0.    0.
     0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.]
 [  

格式信息的组成为                       ：纠错标识+掩码标识+BCH纠错码

In [5]:
import numpy as np
import cv2

image=cv2.imread("./qrcode.png")
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#_,gray=cv2.threshold(gray,100,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
#_,gray=cv2.threshold(gray,160,255,cv2.THRESH_BINARY_INV)
img,contours,hierachy=cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#print contours

for i in range(len(hierachy)):
    if (hierachy[i][2]

cv2.drawContours(image,contours[1],-1,(0,0,255),3)  
  
cv2.imshow("img", image)  
cv2.waitKey(0)

[array([[[  0,   0]],

       [[  0, 440]],

       [[440, 440]],

       [[440,   0]]], dtype=int32), array([[[233, 396]],

       [[234, 395]],

       [[251, 395]],

       [[252, 396]],

       [[252, 404]],

       [[251, 405]],

       [[234, 405]],

       [[233, 404]]], dtype=int32), array([[[314, 387]],

       [[315, 386]],

       [[332, 386]],

       [[333, 387]],

       [[333, 395]],

       [[350, 395]],

       [[350, 387]],

       [[351, 386]],

       [[359, 386]],

       [[360, 387]],

       [[360, 395]],

       [[368, 395]],

       [[369, 396]],

       [[369, 404]],

       [[368, 405]],

       [[324, 405]],

       [[323, 404]],

       [[323, 396]],

       [[315, 396]],

       [[314, 395]]], dtype=int32), array([[[332, 378]],

       [[333, 377]],

       [[341, 377]],

       [[342, 378]],

       [[342, 386]],

       [[341, 387]],

       [[333, 387]],

       [[332, 386]]], dtype=int32), array([[[287, 369]],

       [[288, 368]],

       [[305, 368]]

-1

In [79]:
# -*- coding=utf-8 -*-
import os
import cv2
import numpy as np
import copy
img_path='images'
img_result='results'
def reshape_image(image):
    '''归一化图片尺寸：短边400，长边不超过800，短边400，长边超过800以长边800为主'''
    width,height=image.shape[1],image.shape[0]
    min_len=width
    scale=width*1.0/400
    new_width=400
    new_height=int(height/scale)
    if new_height>800:
        new_height=800
        scale=height*1.0/800
        new_width=int(width/scale)
    out=cv2.resize(image,(new_width,new_height))
    return out
def detecte(image):
    '''提取所有轮廓'''
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #_,gray=cv2.threshold(gray,100,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    _,gray=cv2.threshold(gray,160,255,cv2.THRESH_BINARY_INV)
    #cv2.imshow('img', gray)
    #cv2.waitKey(0)
    img,contours,hierachy=cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    return image,contours,hierachy

def compute_1(contours,i,j):
    '''最外面的轮廓和子轮廓的比例'''
    area1 = cv2.contourArea(contours[i])
    area2 = cv2.contourArea(contours[j])
    if area2==0:
        return False
    ratio = area1 * 1.0 / area2
    if abs(ratio - 49.0 / 25):
        return True
    return False
def compute_2(contours,i,j):
    '''子轮廓和子子轮廓的比例'''
    area1 = cv2.contourArea(contours[i])
    area2 = cv2.contourArea(contours[j])
    if area2==0:
        return False
    ratio = area1 * 1.0 / area2
    if abs(ratio - 25.0 / 9):
        return True
    return False
def compute_center(contours,i):
    '''计算轮廓中心点'''
    M=cv2.moments(contours[i])
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    return cx,cy
def detect_contours(vec):
    '''判断这个轮廓和它的子轮廓以及子子轮廓的中心的间距是否足够小'''
    distance_1=np.sqrt((vec[0]-vec[2])**2+(vec[1]-vec[3])**2)
    distance_2=np.sqrt((vec[0]-vec[4])**2+(vec[1]-vec[5])**2)
    distance_3=np.sqrt((vec[2]-vec[4])**2+(vec[3]-vec[5])**2)
    if sum((distance_1,distance_2,distance_3))/3<3:
        return True
    return False
def juge_angle(rec):
    '''判断寻找是否有三个点可以围成等腰直角三角形'''
    if len(rec)<3:
        return -1,-1,-1
    for i in range(len(rec)):
        for j in range(i+1,len(rec)):
            for k in range(j+1,len(rec)):
                distance_1 = np.sqrt((rec[i][0] - rec[j][0]) ** 2 + (rec[i][1] - rec[j][1]) ** 2)
                distance_2 = np.sqrt((rec[i][0] - rec[k][0]) ** 2 + (rec[i][1] - rec[k][1]) ** 2)
                distance_3 = np.sqrt((rec[j][0] - rec[k][0]) ** 2 + (rec[j][1] - rec[k][1]) ** 2)
                if abs(distance_1-distance_2)<5:
                    if abs(np.sqrt(np.square(distance_1)+np.square(distance_2))-distance_3)<5:
                        return i,j,k
                elif abs(distance_1-distance_3)<5:
                    if abs(np.sqrt(np.square(distance_1)+np.square(distance_3))-distance_2)<5:
                        return i,j,k
                elif abs(distance_2-distance_3)<5:
                    if abs(np.sqrt(np.square(distance_2)+np.square(distance_3))-distance_1)<5:
                        return i,j,k
    return -1,-1,-1
def find(image,image_name,contours,hierachy,root=0):
    '''找到符合要求的轮廓'''
    rec=[]
    #print("hierachy len:", len(hierachy))
    for i in range(len(hierachy)):
        child = hierachy[i][2]
        child_child=hierachy[child][2]
        #print("child:",child,"hierachy[child][2]:",hierachy[child][2])
        if child!=-1 and hierachy[child][2]!=-1:#子类和子类的子类都存在
            if compute_1(contours, i, child) and compute_2(contours,child,child_child):
                cx1,cy1=compute_center(contours,i)
                cx2,cy2=compute_center(contours,child)
                cx3,cy3=compute_center(contours,child_child)
                #print "contours", contours[i], "i", i, "child", child, "child_child",child_child
                #print "1:", cx1, cy1, "2:", cx2, cy2, "3:", cx3, cy3 
                if detect_contours([cx1,cy1,cx2,cy2,cx3,cy3]):
                    rec.append([cx1,cy1,cx2,cy2,cx3,cy3,i,child,child_child])
    '''计算得到所有在比例上符合要求的轮廓中心点'''
    i,j,k=juge_angle(rec)
    #print "i=", i, "j=", j,"k=", k
    if i==-1 or j== -1 or k==-1:
        return
    ts = np.concatenate((contours[rec[i][6]], contours[rec[j][6]], contours[rec[k][6]]))
    rect = cv2.minAreaRect(ts)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    result=copy.deepcopy(image)
    #cv2.drawContours(result, [box], 0, (0, 0, 255), 2)
    #cv2.drawContours(image,contours,rec[i][6],(255,0,0),2)
    #cv2.drawContours(image,contours,rec[j][6],(255,0,0),2)
    #cv2.drawContours(image,contours,rec[k][6],(255,0,0),2)
    #cv2.imshow('img1',image)
    #cv2.waitKey(0)
    #cv2.imshow('img2',result)
    #cv2.waitKey(0)
    #print "box" ,box
    #print "rec", rec[i][6], rec[j][6], rec[k][6]
    #print contours
    return (box, rec[i][6], rec[j][6], rec[k][6])
if __name__ == '__main__':
    image=cv2.imread("./qrcode.png")
    print(image.shape) 
    #image=reshape_image(image)
    #print(image.shape)

    image,contours,hierachy=detecte(image)
    #print("contours:" , hierachy)
    #cv2.drawContours(image2,contours[59],-1,(0,0,255),3)  
  
    cv2.imshow("img", image)  
    cv2.waitKey(0)
    #print "h" ,hierachy
    (box, l1,l2, l3) = find(image,file,contours,np.squeeze(hierachy))
    #cv2.drawContours(image,contours[l3],-1,(0,0,255),3)
    #cv2.imshow("img", image)  
    #cv2.waitKey(0)
    #gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #print len(contours)
    #print contours
    #contours2 = np.delete(contours, 0, axis=0)
    print contours[l2]
    print contours[l1][0][0][1]
    print contours[l2][0][0][1]
    print contours[l3][0][0][0]
    cv2.rectangle(image, (contours[l1][0][0][0],contours[l1][0][0][1]),
                         (contours[l1][2][0][0],contours[l1][2][0][1]),
                         (255,255,255), -1)
    
    cv2.rectangle(image, (contours[l2][0][0][0],contours[l2][0][0][1]),
                         (contours[l2][2][0][0],contours[l2][2][0][1]),
                         (255,255,255), -1)
    
    cv2.rectangle(image, (contours[l3][0][0][0],contours[l3][0][0][1]),
                         (contours[l3][2][0][0],contours[l3][2][0][1]),
                         (255,255,255), -1)
    

    y_start = contours[l3][0][0][0]
    y_end   = contours[l2][3][0][0]
    x_start = contours[l2][0][0][1]
    x_end   = contours[l1][1][0][1]
    print y_start, y_end, x_start, x_end
    
    
    
    image_cut = image[y_start : y_end+1, x_start : x_end+1, :]
    print image_cut.shape
    #cv2.imshow("img", image_cut)  
    #cv2.waitKey(0)
    
    contours2 = contours
    del contours[l1]
    del contours[l2-1]
    del contours[l3-2]

    #cv2.drawContours(image,contours,-1,(0,0,255),3)
    cv2.imshow("img", image)  
    cv2.waitKey(0)
    print len(contours),len(contours2)
    

(441, 441, 3)
[[[342  36]]

 [[342  98]]

 [[404  98]]

 [[404  36]]]
342
36
36
36 404 36 404
(369, 369, 3)
115 115
